In [53]:
val df = spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("subscribe", "spyData")
  .load()
df.printSchema

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



df: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]


In [2]:
val selectDF = df.select(get_json_object(($"value").cast("string"),"$.data.p").alias("price").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.T").alias("ticker").cast("String"),
                         get_json_object(($"value").cast("string"),"$.data.x").alias("exchnage_id").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.s").alias("trade_size").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.t").alias("date_time").cast("Long"))


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [54]:
val temp = selectDF.na.drop()
val temp2 = temp.withColumn("date_time",(col("date_time")/1000000000).cast("timestamp"))
temp2.show

+-----+------+-----------+----------+--------------------+
|price|ticker|exchnage_id|trade_size|           date_time|
+-----+------+-----------+----------+--------------------+
|  304|   SPY|         15|        10|2020-06-24 18:09:...|
|  305|   SPY|         15|       100|2020-06-24 18:10:...|
|  305|   SPY|          2|         1|2020-06-24 18:10:...|
|  305|   SPY|         15|       100|2020-06-24 18:10:...|
|  305|   SPY|         17|       100|2020-06-24 18:11:...|
|  305|   SPY|         15|        10|2020-06-24 18:11:...|
|  305|   SPY|         15|       100|2020-06-24 18:11:...|
|  305|   SPY|         15|       100|2020-06-24 18:11:...|
|  305|   SPY|         17|        10|2020-06-24 18:19:...|
|  305|   SPY|         17|        21|2020-06-24 18:19:...|
|  305|   SPY|         17|        21|2020-06-24 18:19:...|
|  305|   SPY|         17|       100|2020-06-24 18:19:...|
|  305|   SPY|         17|       100|2020-06-24 18:19:...|
|  305|   SPY|         17|        14|2020-06-24 18:19:..

temp: org.apache.spark.sql.DataFrame = [price: int, ticker: string ... 3 more fields]
temp2: org.apache.spark.sql.DataFrame = [price: int, ticker: string ... 3 more fields]


In [55]:
val groupBy_exchangeID = temp2.select(col("price"),col("exchnage_id"),col("date_time"))
                        .groupBy("exchnage_id").count
groupBy_exchangeID.show()

+-----------+-----+
|exchnage_id|count|
+-----------+-----+
|          3|   14|
|         15|   20|
|         17|   45|
|          9|    3|
|          2|   25|
+-----------+-----+



groupBy_exchangeID: org.apache.spark.sql.DataFrame = [exchnage_id: int, count: bigint]
